In [1]:
pip install -U pandasql

Requirement already up-to-date: pandasql in c:\users\crie072\anaconda3\lib\site-packages (0.7.3)
Note: you may need to restart the kernel to use updated packages.


pip install -U xlwt

pip install -U openpyxl

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import csv
import xlrd
import io
from zipfile import ZipFile
from datetime import date, timedelta, time, datetime

from pandasql import sqldf

from pandas.tseries.offsets import MonthEnd

In [3]:
today_yyyymmdd = date.today().strftime("%Y%m%d")
print(today_yyyymmdd)

20201003


In [4]:
#pandasの表示列数を増やす
import pandas as pd
pd.set_option('display.max_columns', 100)

In [6]:
data_dir = Path(r"C:\Users\crie072\Documents\01_データ関連\Adhoc\202010_Applepay")

#input
input_file = "20201001Apple+Pay利用実績（詳細版）.xlsx"

In [6]:
#シート名取得
bk = pd.ExcelFile(data_dir/f"{input_file}")
print(bk.sheet_names)

['Page1_1', 'Page1_2', 'Page1_3', 'Page1_4', 'Page1_5', 'Page1_6', 'Page1_7', 'Page1_8', 'Page1_9', 'Page1_10', 'Page1_11', 'Page1_12', 'Page1_13', 'Page1_14', 'Page1_15', 'Page1_16', 'Page1_17', 'Page1_18', 'Page1_19', 'Page1_20', 'Page1_21', 'Page1_22', 'Page1_23', 'Page1_24', 'Page1_25', 'Page1_26', 'Page1_27', 'Page1_28', 'Page1_29', 'Page1_30', 'Page1_31', 'Page1_32', 'Page1_33', 'Page1_34', 'Page1_35', 'Page1_36', 'Page1_37', 'Page1_38', 'Page1_39', 'Page1_40', 'Page1_41', 'Page1_42', 'Page1_43', 'Page1_44', 'Page1_45', 'Page1_46', 'Page1_47', 'Page1_48', 'Page1_49', 'Page1_50', 'Page1_51', 'Page1_52', 'Page1_53', 'Page1_54', 'Page1_55', 'Page1_56', 'Page1_57', 'Page1_58', 'Page1_59']


In [10]:
#全シート読込
li = []
 
for i in range(0,59):
    df = pd.read_excel(data_dir/f"{input_file}", sheet_name = i)
    li.append(df)
df_all = pd.concat(li, axis=0, ignore_index=True)

In [11]:
df_all

,カード利用年月,加盟店屋号（漢字）,加盟店屋号（カナ）,加盟店業種コード,代表顧客番号,カード申込番号,集計用件数,カード売上合計額
0,202007,処おおぎ 新市役所前店／ｉＤ,ﾒﾝﾄﾞｺﾛｵｵｷﾞｼﾝｼﾔｸｼﾖﾏｴﾃ,3701,155226605873,298098767,1,1070
1,202007,あい整骨院 玉造／ｉＤ,ｱｲｾｲｺﾂｲﾝ ﾀﾏﾂｸﾘ/ID,3701,155223873144,290861158,1,480
2,202007,あおい書店 池尻大橋店／ｉＤ,ｱｵｲｼﾖﾃﾝｲｹｼﾞﾘｵｵﾊｼﾃﾝ/I,1001,155228425478,305534260,1,1655
3,202007,あおい書店四谷三丁目店／ｉＤ,ｱｵｲｼﾖﾃﾝ ﾖﾂﾔｻﾝﾁﾖｳﾒﾃﾝ/,3701,155224186678,323412691,1,1867
4,202007,あしや竹園／ｉＤ,ｱｼﾔ ﾀｹｿﾞﾉ ｱｲﾃﾞｲ,3701,155208842122,296307991,3,13766
...,...,...,...,...,...,...,...,...
58844,202009,ＺＡＲＡ 神戸店／ｉＤ,ｻﾞﾗ ｺｳﾍﾞﾃﾝ/ID,3701,155232645194,326397472,1,9580
58845,202009,ＺＡＲＡ 神戸店／ｉＤ,ｻﾞﾗ ｺｳﾍﾞﾃﾝ/ID,3701,155232682486,326763368,1,21970
58846,202009,ＺＡＲＡ ＨＯＭＥ 青山店／ｉＤ,ｻﾞﾗﾎ-ﾑ ｱｵﾔﾏﾃﾝ/ID,3701,155213574330,297115057,1,1990
58847,202009,Ｚｅｌｌｉｊ／ｉＤ,ｾﾞﾘｰｼﾞﾕ/ID,3701,155216022048,299637818,2,10595


In [13]:
#CSV出力 統合 SASで分析するため
df_all.to_csv(data_dir /'統合_20201001Apple+Pay利用実績.csv', header=True ,index=None, encoding='utf-8-sig')

In [23]:
#SASで属性データをマージしたものを読込
df = pd.read_csv(data_dir /"applepay_userlist.csv",encoding='cp932')
df.head(2)

,カード利用年月,加盟店屋号（漢字）,加盟店屋号（カナ）,加盟店業種コード,代表顧客番号,カード申込番号,集計用件数,カード売上合計額,card,brand,channel,d3,birthday,age,gender,income,income2,prefecture,job,occupation,residense,email_f,web_f,d111,kanyu_date,membership_term,shopping_amt_3month,shopping_amt_6month,shopping_amt_12month,dm_code
0,202007,ヨドバシカメラ マルチメディア梅田／ｉＤ,ﾖﾄﾞﾊﾞｼｶﾒﾗﾏﾙﾁﾒﾃﾞｲｱ/ID,801,1.552000e+11,285532081,1,605,一般,Visa,郵送,19771017,6499,40代,男性,430,05_500万円未満,兵庫県,0,80,1,1,0,20091130,18231,10年以上,0,0,0,Ｄ_神戸店
1,202009,阪急西宮ガーデンズ／ｉＤ,ﾊﾝｷﾕｳﾆｼﾉﾐﾔｶﾞ-ﾃﾞﾝｽﾞ/I,104,1.552000e+11,285532081,2,6150,一般,Visa,郵送,19771017,6499,40代,男性,430,05_500万円未満,兵庫県,0,80,1,1,0,20091130,18231,10年以上,0,0,0,Ｄ_神戸店


In [24]:
df = df.rename(columns={'カード利用年月':'shopping_dt',
                                '加盟店屋号（漢字）':'shop_name',
                                '加盟店屋号（カナ）':'shop_name2',
                                '加盟店業種コード':'shop_code',
                                '代表顧客番号':'daihyouno',
                                'カード申込番号':'applino',
                                '集計用件数':'trans',
                                'カード売上合計額':'shopping_amt'})
df.head(1)

,shopping_dt,shop_name,shop_name2,shop_code,daihyouno,applino,trans,shopping_amt,card,brand,channel,d3,birthday,age,gender,income,income2,prefecture,job,occupation,residense,email_f,web_f,d111,kanyu_date,membership_term,shopping_amt_3month,shopping_amt_6month,shopping_amt_12month,dm_code
0,202007,ヨドバシカメラ マルチメディア梅田／ｉＤ,ﾖﾄﾞﾊﾞｼｶﾒﾗﾏﾙﾁﾒﾃﾞｲｱ/ID,801,1.552000e+11,285532081,1,605,一般,Visa,郵送,19771017,6499,40代,男性,430,05_500万円未満,兵庫県,0,80,1,1,0,20091130,18231,10年以上,0,0,0,Ｄ_神戸店


In [27]:
#利用店舗ランキング
q = """
        SELECT
        shop_name2
        ,age
        ,sum(trans) as no_of_trns
        ,sum(shopping_amt) as sum_shopping_amt
        ,count(distinct daihyouno) as uu
        from 
            df
        group by
        shop_name2,age
        """
df_shop = sqldf(q, locals())
df_shop

,shop_name2,age,no_of_trns,sum_shopping_amt,uu
0,109ｼﾈﾏｽﾞHATｺｳﾍﾞ(ｷﾔﾗｸ,40代,2,1413,2
1,109ｼﾈﾏｽﾞHATｺｳﾍﾞ(ｺﾝｾ),30代,4,3700,4
2,109ｼﾈﾏｽﾞHATｺｳﾍﾞ(ｺﾝｾ),40代,4,5260,4
3,109ｼﾈﾏｽﾞｵｵｻｶEC(ｺﾝｾ)/,20代,1,750,1
4,109ｼﾈﾏｽﾞｵｵｻｶEC(ｺﾝｾ)/,30代,3,2300,3
...,...,...,...,...,...
15886,ﾜﾝﾀﾞ-ﾌﾙ-ﾂ ﾘﾝｸｽｳﾒﾀﾞﾃﾝ,30代,4,1545,1
15887,ﾜﾝﾀﾞ-ﾌﾙ-ﾂ ﾘﾝｸｽｳﾒﾀﾞﾃﾝ,40代,1,990,1
15888,ﾜﾝﾀﾞｰﾌﾞﾗﾂｸﾁｹﾂﾄｳﾘﾊﾞ/I,90代,1,2700,1
15889,ﾜﾝﾀﾞｼﾃｲｼﾈﾏﾄｳ,30代,1,1650,1


In [28]:
q = """
        SELECT
        age
        ,shop_name2
        ,no_of_trns
        ,sum_shopping_amt
        ,uu
        ,row_number() over(partition by age order by no_of_trns desc) as rank
        from 
            df_shop
        order by
        age,rank
        """

df_shop2 = sqldf(q, locals())
df_shop2

,age,shop_name2,no_of_trns,sum_shopping_amt,uu,rank
0,10代,ｾﾌﾞﾝ-ｲﾚﾌﾞﾝ/ID,68,42871,1,1
1,10代,ﾌｱﾐﾘ-ﾏ-ﾄ/ID,19,3659,1,2
2,10代,ﾛ-ｿﾝ/ID,9,3132,1,3
3,10代,ｾｲｺ-ﾏ-ﾄ/ID,7,3538,1,4
4,10代,ﾏｸﾄﾞﾅﾙﾄﾞ/ID,6,1930,1,5
...,...,...,...,...,...,...
15886,90代,ﾕﾆｸﾛ/ID,1,10449,1,49
15887,90代,ﾖｺﾊﾏﾏﾝﾀﾞﾘﾝﾎﾃﾙ/ID,1,5790,1,50
15888,90代,ﾘﾉｱ-ﾗ/ID,1,4400,1,51
15889,90代,ﾛﾂﾎﾟﾝｷﾞｲﾏｶﾂ/ID,1,2650,1,52


In [29]:
q = """
        SELECT*
        from 
            df_shop2
        where 
            rank<=10
        order by
        age,rank
        """

df_shop3 = sqldf(q, locals())
df_shop3

,age,shop_name2,no_of_trns,sum_shopping_amt,uu,rank
0,10代,ｾﾌﾞﾝ-ｲﾚﾌﾞﾝ/ID,68,42871,1,1
1,10代,ﾌｱﾐﾘ-ﾏ-ﾄ/ID,19,3659,1,2
2,10代,ﾛ-ｿﾝ/ID,9,3132,1,3
3,10代,ｾｲｺ-ﾏ-ﾄ/ID,7,3538,1,4
4,10代,ﾏｸﾄﾞﾅﾙﾄﾞ/ID,6,1930,1,5
...,...,...,...,...,...,...
85,90代,ｴﾈｵｽ-ｴｽｴｽ/ID,4,22924,1,6
86,90代,ｼﾖｳﾜｼｴﾙ ｴｽｴｽ/ID,4,25555,1,7
87,90代,ｽｷﾔ/ID,3,2580,1,8
88,90代,ﾄ-ﾎ-ｽﾄｱｶﾐｻﾜﾃﾝ/ID,3,16520,1,9


In [30]:
#CSVアウトプット　利用件数ベースTop20
df_shop3.to_csv(data_dir /'applepay_年代別利用店舗Top10_2007_2009.csv', header=True ,index=None, encoding='shift-jis')

In [ ]:
#Google Pay Apple Pay 登録者数

In [3]:
today_yyyymmdd = date.today().strftime("%Y%m%d")
print(today_yyyymmdd)

20201013


In [5]:
#pandasの表示列数を増やす
import pandas as pd
pd.set_option('display.max_columns', 100)

In [6]:
#Apple Pay
data_dir = Path(r"C:\Users\crie072\Documents\01_データ関連\Adhoc\202010_Applepay")

#input
input_file = "202009applePay抽出.xlsx"

In [7]:
#シート名取得
bk = pd.ExcelFile(data_dir/f"{input_file}")
print(bk.sheet_names)

['Page1_1', 'Page1_2', 'Page1_3']


In [8]:
#全シート読込
li = []
 
for i in range(0,3):
    df = pd.read_excel(data_dir/f"{input_file}", sheet_name = i)
    li.append(df)
df_all = pd.concat(li, axis=0, ignore_index=True)

In [9]:
df_all

,サービスコード,イシュアID,サブイシュアID,データ基準年月日,カード会員識別番号,ステータス,ステータス更新日時,タイプコード,端末識別番号,デバイスタイプ,登録日時,カード会社コード,カード提携方式コード,カード提携会社コード１,カード提携会社コード２,カード種類コード,ブランドコード
0,1,HPF,HPF,20200922,49967841012592040000,UNLINKED,20171206220244,F,2E0161100910172C1F000000000017AC0A01011C748397E8,1,20171009021732,155,10,D00,8,31,1
1,1,HPF,HPF,20200922,49399015191925030000,UNLINKED,20171126002243,F,2E01B9100702031E0800000000000D8B700DA14C0D52D5CD,1,20171123054541,155,10,D00,1,31,1
2,1,HPF,HPF,20200922,49967871948867040000,UNLINKED,20191101000716,F,0450056B664480017263137409943604CA2CF84A7E850051,1,20191006224208,155,10,D00,8,31,1
3,1,HPF,HPF,20200922,49379661923090080000,UNLINKED,20190421003806,F,2E016110090C16001700000000001F91D153B8DC097C754C,1,20190321044419,155,10,D00,8,31,1
4,1,HPF,HPF,20200922,49192003158905020000,UNLINKED,20191111041110,F,045256A3F046800173552007509450131A1DD63C602259F9,1,20191012041042,155,10,D02,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153753,1,HPF,HPF,20200922,49998825938762060000,UNLINKED,20200116080941,F,2E016110070404092D000000000053757286BED449D11A3D,1,20200103155221,155,10,D01,8,31,1
153754,1,HPF,HPF,20200922,49192023404004110000,UNLINKED,20171108044113,F,043022A37A43800172310576339430435FBABDA153EB35CD,1,20171108044101,155,10,D02,1,2,1
153755,1,HPF,HPF,20200922,49767977502974010000,UNLINKED,20180309105508,F,040C1A13F74480017278221579943891CC6F99018859705C,1,20180309010200,155,10,D01,1,31,1
153756,1,HPF,HPF,20200922,49399036968345030000,UNLINKED,20200322004851,F,041C48C3165980019341108017303659B9BE70AB8843CD4C,1,20200321220825,155,10,D00,1,31,1


In [31]:
print(df_all.dtypes)

サービスコード         int64
イシュアID         object
サブイシュアID       object
データ基準年月日        int64
カード会員識別番号      object
ステータス          object
ステータス更新日時       int64
タイプコード         object
端末識別番号         object
デバイスタイプ         int64
登録日時            int64
カード会社コード        int64
カード提携方式コード      int64
カード提携会社コード１    object
カード提携会社コード２     int64
カード種類コード        int64
ブランドコード         int64
dtype: object


In [17]:
print(df_all['ステータス'].unique())

['UNLINKED' 'ACTIVE' 'SUSPENDED']


In [26]:
df_app =df_all[(df_all["ステータス"] == "ACTIVE")]
df_app['ステータス'] = df_app['ステータス'].str.strip()#余白削除
df_app = df_app.rename(columns={'ステータス':'status',
                        'デバイスタイプ':'device_type',
                        'カード会社コード':'card_kaisha_cd',
                        'カード提携方式コード':'card_teikei_code',
                        'カード提携会社コード１':'teikei_cd1',
                        'カード提携会社コード２':'teikei_cd2',
                        'カード種類コード':'card_cd',
                        'ブランドコード':'brand_cd',
                        'カード会員識別番号':'id'})
df_app

C:\Users\crie072\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,サービスコード,イシュアID,サブイシュアID,データ基準年月日,id,status,ステータス更新日時,タイプコード,端末識別番号,device_type,登録日時,card_kaisha_cd,card_teikei_code,teikei_cd1,teikei_cd2,card_cd,brand_cd
6,1,HPF,HPF,20200922,49766577442133010000,ACTIVE,20190728131259,F,044830FB584F800183411445089489874A5D94D40914B1F6,3,20190728131209,155,10,D00,1,2,1
14,1,HPF,HPF,20200922,49525196134845070000,ACTIVE,20190426045340,F,0442362B354D8001826206916394811962B37740A1ABCE87,1,20190426045316,155,10,D00,11,31,1
30,1,HPF,HPF,20200922,49767897970286010000,ACTIVE,20180507024332,F,0444379B8D438001723412421294307353E0858D8739291A,3,20180507024235,155,10,D00,1,31,1
32,1,HPF,HPF,20200922,49767767219747010000,ACTIVE,20190821134251,F,04313063245080018364126153949297581DE808FE229620,1,20190821133550,155,10,D00,1,31,1
38,1,HPF,HPF,20200922,49374041788510080000,ACTIVE,20191120122440,F,044B4CEB0245800172810625089439283F6362B38D34BAB7,1,20190117051746,155,10,D02,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153726,1,HPF,HPF,20200922,49484705133138050000,ACTIVE,20200705001652,F,044A16037F4380017231065474943051E01D932B0A3D9390,1,20200705001557,155,10,D00,8,31,1
153727,1,HPF,HPF,20200922,49078121280071090000,ACTIVE,20200630160752,F,041C2853D7498001810003088994635027D7AD9D5CBD3DA9,1,20200630160725,155,10,D01,1,31,1
153728,1,HPF,HPF,20200922,49967971020739040000,ACTIVE,20191227041009,F,04450FCB0754800191781145903014482E3D8A7B11CE564A,1,20191227040939,155,10,D01,1,31,1
153738,1,HPF,HPF,20200922,49379511999415080000,ACTIVE,20190620044608,F,2E0161100B0C0B07170000000000A70F34CE3B67F842F723,1,20190620044526,155,10,D00,11,31,1


In [27]:
print(df_app['device_type'].unique())

[3 1]


In [28]:
print(df_app['teikei_cd1'].unique())

['D00' 'D02' 'D01']


In [35]:
#apple id
q = """
        SELECT
        'ApplePay' as cate
        ,case 
            when device_type = 2 then "2_tablet"
            when device_type = 3 then "3_watch"
            else "1_phone"
            end as device
        ,case 
            when teikei_cd1 = "D02" then "外商"
            when teikei_cd2 = 5  then "ハウスゴールド"
            when card_cd = 2  then "ゴールド"
            when card_cd = 31 then "一般"
            else "その他"
            end as card
        ,case
            when brand_cd = 2 then "Master" else "Visa"
            end as brand
        ,count(distinct id) as no_of_active
        from 
            df_app
        group by
        1,2,3,4
        """
df_apple = sqldf(q, locals())
df_apple

,cate,device,card,brand,no_of_active
0,ApplePay,1_phone,ゴールド,Visa,1970
1,ApplePay,1_phone,一般,Visa,10277
2,ApplePay,1_phone,外商,Visa,3303
3,ApplePay,3_watch,ゴールド,Visa,238
4,ApplePay,3_watch,一般,Visa,820
5,ApplePay,3_watch,外商,Visa,460


In [36]:
#CSV出力 統合 SASで分析するため
df_apple.to_csv(data_dir /'ApplePay_activeユーザー数.csv', header=True ,index=None, encoding='utf-8-sig')

In [37]:
#Google Pay
data_dir = Path(r"C:\Users\crie072\Documents\01_データ関連\Adhoc\202010_Applepay")

#input
input_file = "202009GooglePay抽出.xlsx"

In [38]:
#シート名取得
bk = pd.ExcelFile(data_dir/f"{input_file}")
print(bk.sheet_names)

['Page1_1']


In [40]:
#全シート読込
li = []
 
for i in range(0,1):
    df = pd.read_excel(data_dir/f"{input_file}", sheet_name = i)
    li.append(df)
df_all = pd.concat(li, axis=0, ignore_index=True)

In [41]:
df_all

,サービスコード,イシュアID,サブイシュアID,データ基準年月日,Token（上３桁）,Token識別番号,Token有効期限,カード番号（上３桁）,カード会員識別番号,カード番号有効期限,ステータス,ステータス更新日時,タイプコード,端末識別番号,デバイスタイプ,登録日時,保証レベル,権限レベル,カード会社コード,カード提携方式コード,カード提携会社コード１,カード提携会社コード２,カード種類コード,ブランドコード
0,2,HPF,0,20200922,0,69725016818231160000,2506,0,49078931032671090000,2105,ACTIVE,20200613101547,F,NaN,1,20200613101547,,,155,10,D00,1,31,1
1,2,HPF,0,20200922,0,69684505140432320000,2506,0,49399125990216120000,2504,DELETE,20200727090518,F,NaN,1,20200613135551,,,155,10,D00,1,31,1
2,2,HPF,0,20200922,0,69522315160003830000,2508,0,49197673132809020000,2306,ACTIVE,20200821120900,F,NaN,1,20200821120900,,,155,10,D00,1,31,1
3,2,HPF,0,20200922,0,69684598140432230000,2504,0,49073401491159090000,2305,DELETE,20200531220022,F,NaN,1,20200418144129,,,155,10,D02,1,2,1
4,2,HPF,0,20200922,0,69198599520168240000,2503,0,49993125839884060000,2407,ACTIVE,20200325160421,F,NaN,1,20200325160421,,,155,10,D02,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312,2,HPF,0,20200922,0,69453456247867860000,2503,0,49379661762431170000,2408,DELETE,20200430164809,F,NaN,1,,,,155,10,D00,1,31,1
313,2,HPF,0,20200922,0,69235509754210370000,2507,0,49767928200204010000,2109,DELETE,20200824220037,F,NaN,1,20200724165324,,,155,10,D01,1,31,1
314,2,HPF,0,20200922,0,69522392160003560000,2503,0,49078941973146090000,2101,DELETE,20200325223657,F,NaN,1,,,,155,10,D00,2,31,1
315,2,HPF,0,20200922,0,69967683967860100000,2507,0,49399075663576030000,2304,DELETE,20200728084222,F,NaN,1,,,,155,10,D00,1,31,1


In [42]:
print(df_all.dtypes)

サービスコード          int64
イシュアID          object
サブイシュアID         int64
データ基準年月日         int64
Token（上３桁）       int64
Token識別番号       object
Token有効期限        int64
カード番号（上３桁）       int64
カード会員識別番号       object
カード番号有効期限        int64
ステータス           object
ステータス更新日時        int64
タイプコード          object
端末識別番号         float64
デバイスタイプ          int64
登録日時            object
保証レベル           object
権限レベル           object
カード会社コード         int64
カード提携方式コード       int64
カード提携会社コード１     object
カード提携会社コード２      int64
カード種類コード         int64
ブランドコード          int64
dtype: object


In [43]:
print(df_all['ステータス'].unique())

['ACTIVE   ' 'DELETE   ' 'ID&VREQ  ']


In [44]:
df_all['ステータス'] = df_all['ステータス'].str.strip()#余白削除

In [51]:
df_gg =df_all[(df_all["ステータス"] == "ACTIVE")]
df_gg['ステータス'] = df_gg['ステータス'].str.strip()#余白削除
df_gg = df_gg.rename(columns={'ステータス':'status',
                        'デバイスタイプ':'device_type',
                        'カード会社コード':'card_kaisha_cd',
                        'カード提携方式コード':'card_teikei_code',
                        'カード提携会社コード１':'teikei_cd1',
                        'カード提携会社コード２':'teikei_cd2',
                        'カード種類コード':'card_cd',
                        'ブランドコード':'brand_cd',
                        'カード会員識別番号':'id'})
df_gg

C:\Users\crie072\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,サービスコード,イシュアID,サブイシュアID,データ基準年月日,Token（上３桁）,Token識別番号,Token有効期限,カード番号（上３桁）,id,カード番号有効期限,status,ステータス更新日時,タイプコード,端末識別番号,device_type,登録日時,保証レベル,権限レベル,card_kaisha_cd,card_teikei_code,teikei_cd1,teikei_cd2,card_cd,brand_cd
0,2,HPF,0,20200922,0,69725016818231160000,2506,0,49078931032671090000,2105,ACTIVE,20200613101547,F,NaN,1,20200613101547,,,155,10,D00,1,31,1
2,2,HPF,0,20200922,0,69522315160003830000,2508,0,49197673132809020000,2306,ACTIVE,20200821120900,F,NaN,1,20200821120900,,,155,10,D00,1,31,1
4,2,HPF,0,20200922,0,69198599520168240000,2503,0,49993125839884060000,2407,ACTIVE,20200325160421,F,NaN,1,20200325160421,,,155,10,D02,1,2,1
7,2,HPF,0,20200922,0,69368205617104000000,2508,0,49967711411717040000,2507,ACTIVE,20200810171650,F,NaN,1,20200810171650,,,155,10,D00,11,31,1
8,2,HPF,0,20200922,0,69278883425035810000,2504,0,49525126804433070000,2204,ACTIVE,20200420132512,F,NaN,1,20200420132512,,,155,10,D00,1,31,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,2,HPF,0,20200922,0,69557880390637790000,2507,0,49525136169142070000,2102,ACTIVE,20200717230838,F,NaN,1,20200717230838,,,155,10,D00,2,31,1
304,2,HPF,0,20200922,0,69167654511402130000,2507,0,49394545894824030000,2101,ACTIVE,20200705004642,F,NaN,1,20200705004642,,,155,10,D02,1,2,1
305,2,HPF,0,20200922,0,69453458247867680000,2503,0,49998765944290060000,2412,ACTIVE,20200325114148,F,NaN,1,20200325114148,,,155,10,D00,1,31,1
306,2,HPF,0,20200922,0,69522314160003920000,2508,0,49398855136977030000,2507,ACTIVE,20200809094901,F,NaN,1,20200809094901,,,155,10,D00,1,2,1


In [52]:
print(df_gg['device_type'].unique())

[1]


In [53]:
print(df_gg['teikei_cd1'].unique())

['D00' 'D02' 'D01']


In [54]:
#Google id
q = """
        SELECT
        'GooglePay' as cate
        ,case 
            when device_type = 2 then "2_tablet"
            when device_type = 3 then "3_watch"
            else "1_phone"
            end as device
        ,case 
            when teikei_cd1 = "D02" then "外商"
            when teikei_cd2 = 5  then "ハウスゴールド"
            when card_cd = 2  then "ゴールド"
            when card_cd = 31 then "一般"
            else "その他"
            end as card
        ,case
            when brand_cd = 2 then "Master" else "Visa"
            end as brand
        ,count(distinct id) as no_of_active
        from 
            df_gg
        group by
        1,2,3,4
        """
df_google = sqldf(q, locals())
df_google

,cate,device,card,brand,no_of_active
0,GooglePay,1_phone,ゴールド,Visa,33
1,GooglePay,1_phone,一般,Visa,127
2,GooglePay,1_phone,外商,Visa,49


In [56]:
#CSV出力
df_google.to_csv(data_dir /'Google_activeユーザー数.csv', header=True ,index=None, encoding='utf-8-sig')